In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya

/home/ubuntu/dev/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/ubuntu/dev/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
from bs4 import BeautifulSoup
import requests

In [4]:
def crawl_dbp(word):
    url = f'https://prpm.dbp.gov.my/Cari1?keyword={word}'
    r = requests.get(url)
    soup = BeautifulSoup(r._content)
    definitions = soup.select("div[class*=tab-pane]")
    definitions = [d.text for d in definitions]
    spans = soup.find_all('span', id = 'MainContent_SearchInfoTesaurus_lblTesaurus')
    if len(spans):
        tesaurus = spans[0]
        tds = tesaurus.findAll('td')
        selected_td = None
        for td in tds:
            if 'javascript:showModalDialog' in str(td):
                selected_td = td
        if selected_td:
            tesaurus = [a.text for a in selected_td.find_all('a')]
        else:
            tesaurus = None
    else:
        tesaurus = None
    return {'definisi': definitions, 'tesaurus': tesaurus}

In [5]:
%%time

crawl_dbp('agung')

CPU times: user 8.53 ms, sys: 2.97 ms, total: 11.5 ms
Wall time: 207 ms


{'definisi': [], 'tesaurus': None}

In [6]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [7]:
import mp
import os
import json
from tqdm import tqdm
import traceback
import redis

In [8]:
db = redis.Redis(host='localhost', port=6379, db=10)

In [16]:
len(db.keys())

811354

In [10]:
def loop(words):
    words, _ = words
    db = redis.Redis(host='localhost', port=6379, db=10)
    for word in tqdm(words):
        if word in db:
            continue
        try:
            r = crawl_dbp(word)
            db.set(word, json.dumps(r))
        except Exception as e:
            print(e)
            # print(traceback.format_exc())

In [11]:
with open('unique-words.json') as fopen:
    words = json.load(fopen)
    
len(words)

798883

In [12]:
words[:10]

['a.m.n.',
 'aa',
 'aa-al',
 'aa-ee',
 'aa-side',
 'aa-stein',
 'aa-thondaimaan',
 'aa-tondai',
 'aaa',
 'aaaa']

In [13]:
loop((words[:5], 0),)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 9098.27it/s]


In [ ]:
mp.multiprocessing(words, loop, cores = 50, returned = False)

 52%|███████████████████████████████████████████▍                                        | 8269/15977 [15:45<1:30:15,  1.42it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

